## Libraries

In [10]:
!pip install numpy
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install skll

import matplotlib.pyplot as plt

import sklearn.metrics as sklm
import skll.metrics as skllm
import copy
import json

import pandas as pd
import numpy as np

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Utils

In [11]:
PATH = "./"
ROOT_PATH = "../../"

In [12]:
def retrieve_data(flow_interval):
    path = "{0}dataset/dataset_flow_{1}.csv".format(ROOT_PATH, flow_interval)
    print(PATH)
    data = pd.read_csv(path, ';')
    
    data['Flow'].apply(int)
    data['AveSpeed'].apply(float)
    data['Density'].apply(float)
    data['Sunday'].apply(int)
    data['Monday'].apply(int)
    data['Tuesday'].apply(int)
    data['Wednesday'].apply(int)
    data['Thursday'].apply(int)
    data['Friday'].apply(int)
    data['Saturday'].apply(int)
      
    return data

In [13]:
def print_json (obj):
  print(json.dumps(obj, sort_keys=True, indent=4))

In [14]:
def store(obj, path, name):
  with open("{0}{1}/{2}.json".format(PATH, path, name), 'w') as json_file:
    json.dump(obj, json_file, sort_keys=True, indent=4)

## Testing

In [15]:
def plot_prediction (Y, Y_hat, title, alpha, gamma):
  """ Plot Prediction
  
  Plot the prediction (Flow x Time) of what was expected and what
  was predicted.
  """
  alpha_dec = str(alpha - int(alpha)).split('.')[1]
  gamma_dec = str(gamma - int(gamma)).split('.')[1]
  name = f"{title}_alpha_d{alpha_dec}_gamma_d{gamma_dec})"
  path = f"plots/prediction/{name}"
    
  plt.plot(Y)
  plt.plot(Y_hat)
  plt.title(f"Predição do Modelo {title} (alpha {alpha}, gamma {gamma})")
  plt.ylabel('Fluxo')
  plt.xlabel('Tempo')
  plt.legend(['esperado', 'observado'], loc='upper left')
  plt.rcdefaults()

  plt.savefig(path + ".png", bbox_inches='tight')
  # plt.savefig(path + ".pdf")

  plt.close('all')

In [16]:
# Model Parameters
FLOW_INTERVAL = 150 # the interval size for each flow
ALPHAS, GAMMAS = [], []
for i in range(1001):
    ALPHAS.append(i/1000)
GAMMAS = ALPHAS

In [17]:
def double_exponential_smoothing(X, alpha, gamma):
  global result_data
  
  name = "DES"
  expected, observed = [], []
  W = 1

  X_test = np.array_split(X, 10)
  F = X_test[8][-1]
  Lm1 = F
  bm1 = 0
  for x in X_test[9]:
    L = (alpha * x) + ( (1 - alpha) * (Lm1 + bm1) )
    b = ( gamma * (L - Lm1) ) + ( (1 - gamma) * bm1 )
    F1 = L + (b*W)
    
    expected.append(int(x))
    observed.append(float(F))
    F = F1
    Lm1 = L
    bm1 = b
  
  result_data['results']['RMSE'] = sklm.mean_squared_error(expected, observed, squared=False)
  result_data['results']['MAE'] = sklm.mean_absolute_error(expected, observed)
  result_data['results']['Kappa'] = skllm.kappa(expected, observed)
  result_data['results']['expected'] = expected
  result_data['results']['observed'] = observed
  alpha_dec = str(alpha - int(alpha)).split('.')[1]
  gamma_dec = str(gamma - int(gamma)).split('.')[1]
  store(result_data, "results", f"{name}_alpha_d{alpha_dec}_gamma_d{gamma_dec}")
  # store(result_data, "results", "{0}_alpha_d{1}_gamma_d{2}".format(name, alpha_dec, gamma_dec))

In [18]:
global result_data

data = retrieve_data(FLOW_INTERVAL)
sequence = np.array(data['Flow'])

for alpha, gamma in [(x,y) for x in ALPHAS for y in GAMMAS]:
  print("ALPHA: {0}".format(alpha))
  print("GAMMA: {0}".format(gamma))
  result_data = {
    'results': {},
    'meta': {
      'FLOW_INTERVAL': FLOW_INTERVAL,
    }
  }
  
  double_exponential_smoothing(sequence, alpha, gamma)
  print("RMSE: {0}".format(result_data['results']['RMSE']))
  print("MAE: {0}".format(result_data['results']['MAE']))
  print("Kappa: {0}".format(result_data['results']['Kappa']))
  
  expected = result_data['results']['expected']
  observed = result_data['results']['observed']
  plot_prediction(expected, observed, "DES", alpha, gamma)
  print("####################################################")

./
ALPHA: 0.001
GAMMA: 0.001
RMSE: 9.089171078193255
MAE: 7.85322749317407
Kappa: -0.001569970485579164
####################################################
ALPHA: 0.001
GAMMA: 0.01
RMSE: 9.201481282623503
MAE: 8.059864038395009
Kappa: -0.0024293642924422265
####################################################
ALPHA: 0.001
GAMMA: 0.1
RMSE: 31.942577088252037
MAE: 28.10854539088203
Kappa: -0.0008745944989130283
####################################################
ALPHA: 0.001
GAMMA: 0.2
RMSE: 13.910295736312252
MAE: 11.35705448046948
Kappa: 0.0054899406472536105
####################################################
ALPHA: 0.001
GAMMA: 0.5
RMSE: 26.108981024358357
MAE: 22.0665205955578
Kappa: -0.0007303720833804483
####################################################
ALPHA: 0.001
GAMMA: 0.8
RMSE: 10.262390869515714
MAE: 8.074102084899131
Kappa: 0.014155811783637007
####################################################
ALPHA: 0.01
GAMMA: 0.001
RMSE: 7.9062471813051145
MAE: 6.665065870100392